<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_12/s12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download our code repository and install python dependencies

In [ ]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git pull
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git pull
!pip install --quiet "torchinfo" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts"
!pip install --quiet "prettytable"
!pip install --quiet "torch_lr_finder"
!pip install --quiet "grad-cam"

Add all the imports:

In [ ]:
from __future__ import print_function
import os
import sys
from typing import Any
sys.path.append("../dl_hub/models")
sys.path.append("../ERA_V1/session_12")
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader
import albumentations as A
import albumentations.augmentations as AA
from albumentations.pytorch import ToTensorV2
from dl_hub.models.custom_resnet import CustomResnet, model_summary

Lightning related imports

In [ ]:
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, LightningDataModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy

Set up environment and add random seed

In [ ]:
GLOBAL_SEED = 42
seed_everything(GLOBAL_SEED)
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 512 if torch.cuda.is_available() else 64
NUM_WORKERS = 2

Define data module

In [ ]:
from dl_hub.dataloaders.pl_custom_cifar10_datamodule import CustomCifar10DataModule
train_transforms = A.Compose([
    A.Normalize(mean=(0.491,0.482,0.447),std=(0.247,0.244,0.262), always_apply=True),
    A.transforms.PadIfNeeded(min_height=40, min_width=40, always_apply=True),
    AA.crops.transforms.RandomCrop(height=32, width=32, always_apply=True),
    A.HorizontalFlip(),
    AA.dropout.coarse_dropout.CoarseDropout(max_holes = 1,
                                            max_height=8,
                                            max_width=8,
                                            min_holes = 1,
                                            min_height=8,
                                            min_width=8,
                                            fill_value=(0.491, 0.482, 0.447),
                                            mask_fill_value = None),

                ToTensorV2()])
test_transforms = A.Compose([
    A.Normalize(mean=(0.491,0.482,0.447),std=(0.247,0.244,0.262), always_apply=True),
    ToTensorV2(),])
val_transforms = test_transforms
data_dir = PATH_DATASETS
cifar10_dm = CustomCifar10DataModule(
    data_dir,
    train_transforms,
    val_transforms, 
    test_transforms,
    BATCH_SIZE,
    val_split = 0.1
)
cifar10_dm.prepare_data()
cifar10_dm.setup()

Get Best LR

In [ ]:
from torch_lr_finder import LRFinder
from dl_hub.models.pl_custom_resnet import LitResnet

def find_best_lr(model, train_loader, optimizer, criterion, device):
    lr_finder = LRFinder(model, optimizer, criterion, device)
    lr_finder.range_test(train_loader, end_lr=10, num_iter=200, step_mode='exp')
    lr_finder.plot()
    lr_finder.reset()
    return lr_finder.history

drop_out_probability=0.05
dummy_model = LitResnet(lr=0.01,base_channels=3, num_classes=10,
                  drop_out_probability=drop_out_probability)
tl = DataLoader(cifar10_dm.train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=NUM_WORKERS,
                          shuffle=True)
optimizer = optim.Adam(dummy_model.parameters(),lr=0.01, weight_decay=1e-4)
criterion = F.nll_loss
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
lr_history = find_best_lr(dummy_model, tl, optimizer, criterion, device)

In [ ]:
from dl_hub.utils.utils import LRFinder

drop_out_probability=0.05
criterion = F.nll_loss
dummy_model = LitResnet(criterion, lr=0.01,base_channels=3, num_classes=10,
                  drop_out_probability=drop_out_probability)
tl = DataLoader(cifar10_dm.train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=NUM_WORKERS,
                          shuffle=True)
optimizer = optim.Adam(dummy_model.parameters(),lr=0.01, weight_decay=1e-4)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
lr_history = find_best_lr(dummy_model, tl, optimizer, criterion, device)

Define the model, optimizer and scheduler

In [ ]:
NUM_EPOCHS = 24
drop_out_probability=0.1
criterion = F.nll_loss
resnet_model = LitResnet(criterion, lr=0.01,base_channels=3, num_classes=10,
                  drop_out_probability=drop_out_probability, epochs=NUM_EPOCHS)
resnet_model.set_optimizer(optim.Adam(resnet_model.parameters(),
                               lr=resnet_model.hparams.lr,
                               weight_decay=1e-4))
steps_per_epoch = len(cifar10_dm.train_dataset) // BATCH_SIZE
max_epochs = 24
resnet_model.set_scheduler_dict({
    "scheduler": OneCycleLR(
        resnet_model.optimizer,
        max_lr = 1.87E-2,
        epochs=resnet_model.epochs,
        steps_per_epoch=steps_per_epoch,
        pct_start = 5./resnet_model.epochs,
        div_factor=2000,
        final_div_factor=1,
        three_phase=False,
        anneal_strategy='linear',
        verbose=False),
    "interval": "step",
    })

In [ ]:
resnet_model.configure_optimizers()

In [ ]:
steps_per_epoch = len(cifar10_dm.train_dataset) // BATCH_SIZE
trainer = Trainer(
    max_epochs=resnet_model.epochs,
    max_steps = steps_per_epoch*resnet_model.epochs,
    accelerator='auto',
    devices = 1 if torch.cuda.is_available() else None,
    logger = CSVLogger(save_dir="logs/"),
    log_every_n_steps = 1,
    callbacks=[LearningRateMonitor(logging_interval='step'),
               TQDMProgressBar(refresh_rate=10)],
    num_sanity_val_steps=0)

trainer.fit(resnet_model, cifar10_dm)
trainer.test(resnet_model, datamodule = cifar10_dm)

In [ ]:
import pandas as pd
import seaborn as sn
metrics = pd.read_csv(f"{trainer.logger.log_dir}/metrics.csv")
del metrics["epoch"]
metrics.set_index("step", inplace=True)
display(metrics.dropna(axis=1, how="all").head())
sn.relplot(data=metrics, kind="line")